In [1]:
import os
import pandas as pd
from datetime import datetime

In [23]:
def desagg_cols(df):
    df['UF'] = df['Territorio'].str.split('-').str[0]
    df['Estado'] = df['Territorio'].str.split('-').str[1]
    df.drop(columns=['Unnamed: 0','Territorio'], inplace=True)
    return df

In [3]:
def to_date(df):
    df['data'] = df['ano'].astype(str) + '-' + df['mes'].apply(lambda x: '0' + str(x) if x < 10 else str(x))
    df['data'] = pd.to_datetime(df['data'], format='%Y-%m')

## Carregando e tratando dados da SAGI

In [70]:
files = [f for f in os.listdir("Data/Fam/") if os.path.isfile(os.path.join("Data/Fam/", f))]
total = len(files)
dfs=[]
for file in files:
    df = pd.read_csv(f"Data/Fam/{file}", header=0, index_col='Código')
    df['ano'] = df['Referência'].apply(lambda x: int(x.split('/')[1]))
    df['mes'] = df['Referência'].apply(lambda x: int(x.split('/')[0]))
    df.drop(columns=['Referência'], inplace=True)
    df.set_index(['UF','Unidade Territorial','ano','mes'], inplace=True)
    dfs.append(df)
    print('Arquivo lido. Restam', total-len(dfs), 'arquivos...')
df = pd.concat(dfs, axis=1)


Arquivo lido. Restam 5 arquivos...
Arquivo lido. Restam 4 arquivos...
Arquivo lido. Restam 3 arquivos...
Arquivo lido. Restam 2 arquivos...
Arquivo lido. Restam 1 arquivos...
Arquivo lido. Restam 0 arquivos...


In [71]:
df.rename(columns={
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_nao_benef_ext_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_benef_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_benef_ext_pobreza',
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_nao_benef_pobreza',
    'Famílias inscritas no Cadastro Único em situação de extrema pobreza':'fam_inscritas_ext_pobreza',
    'Famílias inscritas no Cadastro Único em situação de pobreza':'fam_insc_pobreza',
    },
    inplace = True)

In [72]:
df['demanda_reprimida_total'] = df['fam_nao_benef_pobreza'] + df['fam_nao_benef_ext_pobreza']

In [73]:
uf = df.groupby(['ano','mes','UF']).sum()

In [74]:
br = df.groupby(['ano','mes']).sum()

In [75]:
df.reset_index(inplace=True)
uf.reset_index(inplace=True)
br.reset_index(inplace=True)

In [77]:
uf.tail()

,ano,mes,UF,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total
3154,2022,4,RS,77584.0,79879.0,410019.0,48933.0,487578,128809,126517.0
3155,2022,4,SC,33588.0,38483.0,134067.0,20820.0,167637,59302,54408.0
3156,2022,4,SE,40597.0,20235.0,326343.0,26647.0,366936,46882,67244.0
3157,2022,4,SP,381168.0,318497.0,1813611.0,207151.0,2194717,525648,588319.0
3158,2022,4,TO,14192.0,26589.0,113149.0,16145.0,127325,42734,30337.0


In [78]:
to_date(br)
to_date(uf)
to_date(df)

## Carregando e tratando dados do CECAD

In [79]:
siglas = pd.read_csv('Utils/estados_siglas.csv', usecols=['Sigla', 'Estado'])
uf_pes_cecad = desagg_cols(pd.read_csv('Data/Pessoas/CECAD/UF_CECAD.csv'))
uf_fam_cecad = desagg_cols(pd.read_csv('Data/Fam/CECAD/UF_CECAD.csv'))
br_fam_cecad = pd.read_csv('Data/Fam/CECAD/BR_CECAD.csv')
br_pes_cecad = pd.read_csv('Data/Pessoas/CECAD/BR_CECAD.csv')

In [80]:
max_year = datetime.strptime('2022-05-01', '%Y-%m-%d').year
max_month = datetime.strptime('2022-05-01', '%Y-%m-%d').month

In [61]:
def rename_and_label_date(df, unidade, max_year, max_month, UF):
    if UF:
        df.columns = [f'nao_recebe_{unidade}', f'recebe_{unidade}','sem_resposta', f'total_{unidade}', 'UF', 'Estado']
        df.drop(columns=['Estado', 'sem_resposta'], inplace=True)
    else:
        df.columns = ['sem_nome', 'Territorio',f'nao_recebe_{unidade}', f'recebe_{unidade}','sem_resposta', f'total_{unidade}']
        df.drop(columns=['sem_nome', 'Territorio', 'sem_resposta'], inplace=True)
    df['ano'] = max_year
    df['mes'] = max_month
    return df

In [81]:
uf_pes_cecad = rename_and_label_date(uf_pes_cecad, 'pessoas', max_year, max_month, UF=True)
uf_fam_cecad = rename_and_label_date(uf_fam_cecad, 'familia', max_year, max_month, UF=True)
br_fam_cecad = rename_and_label_date(br_fam_cecad, 'familia', max_year, max_month, UF=False)
br_pes_cecad = rename_and_label_date(br_pes_cecad, 'pessoas', max_year, max_month, UF=False)

In [82]:
uf_cecad = pd.merge(uf_pes_cecad, uf_fam_cecad, on=['UF', 'ano', 'mes'])

In [83]:
br_cecad = pd.merge(br_pes_cecad, br_fam_cecad, on=['ano', 'mes'])

In [84]:
uf.tail()

,ano,mes,UF,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data
3154,2022,4,RS,77584.0,79879.0,410019.0,48933.0,487578,128809,126517.0,2022-04-01
3155,2022,4,SC,33588.0,38483.0,134067.0,20820.0,167637,59302,54408.0,2022-04-01
3156,2022,4,SE,40597.0,20235.0,326343.0,26647.0,366936,46882,67244.0,2022-04-01
3157,2022,4,SP,381168.0,318497.0,1813611.0,207151.0,2194717,525648,588319.0,2022-04-01
3158,2022,4,TO,14192.0,26589.0,113149.0,16145.0,127325,42734,30337.0,2022-04-01


In [85]:
uf_cecad.head()

,nao_recebe_pessoas,recebe_pessoas,total_pessoas,UF,ano,mes,nao_recebe_familia,recebe_familia,total_familia
0,20.117,108.86,128.977,AC,2022,5,41.564,376.376,417.94
1,70.133,479.606,549.739,AL,2022,5,119.227,1.305.110,1.424.337
2,30.548,95.352,125.9,AP,2022,5,56.498,303.133,359.631
3,97.425,484.04,581.465,AM,2022,5,185.317,1.632.821,1.818.138
4,397.526,2.230.554,2.628.080,BA,2022,5,652.937,5.785.555,6.438.492


In [65]:
uf = uf.merge(uf_cecad, on=['UF', 'ano', 'mes'], how='left').merge(siglas, left_on='UF', right_on='Sigla', how='left')
uf.drop(columns=['Sigla'], inplace=True)

In [69]:
uf.tail()

,ano,mes,UF,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,...,recebe_familia_x,total_familia_x,Estado_x,nao_recebe_pessoas_y,recebe_pessoas_y,total_pessoas_y,nao_recebe_familia_y,recebe_familia_y,total_familia_y,Estado_y
3154,2022,4,RS,77584.0,79879.0,410019.0,48933.0,487578,128809,126517.0,...,NaN,NaN,RIO GRANDE DO SUL,NaN,NaN,NaN,NaN,NaN,NaN,RIO GRANDE DO SUL
3155,2022,4,SC,33588.0,38483.0,134067.0,20820.0,167637,59302,54408.0,...,NaN,NaN,SANTA CATARINA,NaN,NaN,NaN,NaN,NaN,NaN,SANTA CATARINA
3156,2022,4,SE,40597.0,20235.0,326343.0,26647.0,366936,46882,67244.0,...,NaN,NaN,SERGIPE,NaN,NaN,NaN,NaN,NaN,NaN,SERGIPE
3157,2022,4,SP,381168.0,318497.0,1813611.0,207151.0,2194717,525648,588319.0,...,NaN,NaN,SÃO PAULO,NaN,NaN,NaN,NaN,NaN,NaN,SÃO PAULO
3158,2022,4,TO,14192.0,26589.0,113149.0,16145.0,127325,42734,30337.0,...,NaN,NaN,TOCANTINS,NaN,NaN,NaN,NaN,NaN,NaN,TOCANTINS


In [66]:
br.head()

,ano,mes,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data
0,2012,8,0.0,0.0,0.0,0.0,12897221,5265743,0.0,2012-08-01
1,2012,9,0.0,0.0,0.0,0.0,13006755,5278170,0.0,2012-09-01
2,2012,10,0.0,0.0,0.0,0.0,13076366,5288902,0.0,2012-10-01
3,2012,11,0.0,0.0,0.0,0.0,13131359,5298295,0.0,2012-11-01
4,2012,12,0.0,0.0,0.0,0.0,13161099,5311337,0.0,2012-12-01


In [67]:
br_cecad

,nao_recebe_pessoas,recebe_pessoas,total_pessoas,ano,mes,nao_recebe_familia,recebe_familia,total_familia
0,3.968.183,17.676.824,21.645.007,2022,5,7.390.728,48.587.348,55.978.076


In [ ]:
br = br.merge(br_cecad, on=['ano', 'mes'], how='left')
uf.drop(columns=['Sigla'], inplace=True)

In [43]:
for col in [col for col in uf.columns if 'pessoas' in col]:
    uf[col] = uf[col].apply(lambda x: float(str(x).replace('.', '')))

In [44]:
dt = datetime.datetime.now()
br['ts'] = datetime.date(dt.year, dt.month, dt.day)

In [48]:
br.to_parquet('Results/fam_br.parquet')
uf.to_parquet('Results/fam_uf.parquet')
df.to_parquet('Results/fam_mun.parquet')